## Using CAP in Julia
Sebastian Gutsche, Sebastian Posur

This notebook demonstrates how to use CAP from julia, via GAP.jl.

We start by initializing GAP

In [1]:
using GAP

Adding path /home/sebastian/Software/GAPJulia/gap/.libs to DL_LOAD_PATH
 ┌───────┐   GAP 4.dev of today
 │  GAP  │   https://www.gap-system.org
 └───────┘   Architecture: x86_64-pc-linux-gnu-julia64-kv6
 Configuration:  gmp 6.1.2, Julia GC, Julia 1.1.0, readline
 Loading the library and packages ...
 Packages:   GAPDoc 1.6.2.dev, PrimGrp 3.3.1, SmallGrp 1.3, TransGrp 2.0.4
 Try '??help' for help. See also '?copyright', '?cite' and '?authors'


In this notebook, we will learn how to use CAP as a computational tool from Julia

## Abelian categories

### Category theory in computer algebra: first steps in CAP

## Introduction

**Categorical abstraction** is a powerful **computational tool**.

Category theory gains its computational power from three key observations:

1. equivalences of categories can transform otherwise inaccessible mathematical objects into computationally easily graspable entities,

2. category theory defines a *language* that respects the isomorphism-invariant structures and properties of mathematical objects, an idea that justifies the treatment of equivalent categories as "being equal from a categorical point of view",

3. lots of mathematicians nowadays use category theory as a widely accepted language to express their ideas, and thus, constructive category theory has a great range of applications.

In this worksheet, we will explore 

- the category of finite dimensional vector spaces $\mathrm{vec}_k$ over a computable field $k$,
- the category of finitely presented (left) modules $\mathrm{mod}_R$ over a computable ring $R$.

We will do this by giving **computational models** for these categories,
i.e., we will construct categories $\mathrm{mat}_k$ and $\mathrm{fpres}_R$
for which we have

$$\mathrm{vec}_k \simeq \mathrm{mat}_k$$

and

$$\mathrm{mod}_R \simeq \mathrm{fpres}_R\text{,}$$

but with the advantage that these categories will be much easier to work with from a computational point of view.

## Computing in abelian categories

The categories $\mathrm{vec}_k$ and $\mathrm{mod}_R$ are both
examples of **abelian categories**.

*Remark for those who already know what an abelian category is:
the fact that $\mathrm{mod}_R$ (which is the
category of *finitely presented* modules and not to be confused with
the category of *all* $R$-modules) is an abelian category relies
on the ring $R$ being coherent, a property that will be implied
by our notion of a computable ring.*

The computational aspects of the *language of abelian categories*
will be our guide in this section.

### Finite dimensional vector spaces

Our computational model $\mathrm{mat}_k$ for finite dimensional vector spaces,
which we call the **matrix category** of $k$, is part of the following $\textsf{GAP}$-package:

In [2]:
try
    GAP.LoadPackageAndExposeGlobals( "LinearAlgebraForCAP", "CAP", all_globals = true )
catch(e)
    GAP.LoadPackageAndExposeGlobals( "LinearAlgebraForCAP", "CAP", all_globals = true )
end

Syntax error: Name used for two locals in /home/sebastian/Software/GAPJulia/ga\
p/pkg/Polycyclic/gap/basic/pcppcps.gi:124
    local coll, rels, n, c, ind, g, d, todo, val, j, f, h, e, a, d, k, b, u, t\
;
                                                                 ^
Syntax warning: Unbound global variable in /home/sebastian/Software/GAPJulia/g\
ap/pkg/Polycyclic/gap/basic/pcpgrps.gi:144
    return SubgroupByIgs( G, AddIgsToIgs( fac, nor ) );
                             ^^^^^^^^^^^
Syntax warning: Unbound global variable in /home/sebastian/Software/GAPJulia/g\
ap/pkg/Polycyclic/gap/basic/pcpsers.gi:133
    cycl := CyclicDecomposition( pcp );
            ^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /home/sebastian/Software/GAPJulia/g\
ap/pkg/Polycyclic/gap/basic/pcpsers.gi:441
    A   := SubgroupByIgs( G, AddIgsToIgs(gens, NumeratorOfPcp( pcps[1] )));
                             ^^^^^^^^^^^
Syntax warning: Unbound global variable in /home/sebastian/Software/GAPJul

First, we need to create a field. For example, we can create the rationals $\mathbb{Q}$:

In [3]:
QQ = CAP.HomalgFieldOfRationals()

─────────────────────────────────────────────────────────────────────────────
Loading  Gauss 2018.09.08 (Extended Gauss functionality for GAP)
by Simon Goertzen (http://wwwb.math.rwth-aachen.de/goertzen).
maintained by:
   Mohamed Barakat (http://wwwb.math.rwth-aachen.de/~barakat/) and
   Sebastian Gutsche (http://wwwb.math.rwth-aachen.de/~gutsche/).
Homepage: http://homalg-project.github.io/homalg_project/Gauss/
─────────────────────────────────────────────────────────────────────────────
─────────────────────────────────────────────────────────────────────────────
Loading  GaussForHomalg 2019.02.01 (Gauss functionality for the homalg project)
by Simon Goertzen (http://wwwb.math.rwth-aachen.de/goertzen).
maintained by:
   Mohamed Barakat (http://www.mathematik.uni-kl.de/~barakat/).
Homepage: http://homalg-project.github.io/homalg_project/GaussForHomalg/
─────────────────────────────────────────────────────────────────────────────


GAP: Q

We could also create a finite field $\mathbb{F}_3$ with $3$ elements:

In [4]:
F3 = CAP.HomalgRingOfIntegers( 3 )

GAP: GF(3)

The following constructor takes such a field as an input and returns
our computational model of the category of finite dimensional vector spaces as an output:

In [5]:
matQQ = CAP.MatrixCategory( QQ )

GAP: Category of matrices over Q

The $\mathsf{GAP}$-object $\mathrm{mat}_{\mathbb{Q}}$ comes with some preset knowledge:

In [6]:
CAP.IsAbelianCategory( matQQ )

true

We claimed in the introduction that we have an equivalence
$$\mathrm{vec}_{\mathbb{Q}} \simeq \mathrm{mat}_{\mathbb{Q}}\text{,}$$
so, let us learn more about $\mathrm{mat}_{\mathbb{Q}}$.
We start with the **objects**. To construct an object in 
$\mathrm{mat}_{\mathbb{Q}}$, we use the following constructor:

In [7]:
V1 = CAP.VectorSpaceObject( 1, QQ )

GAP: A vector space object over Q of dimension 1

Since we expect this object to represent a vector space, we ask for its dimension:

In [8]:
CAP.Dimension( V1 )

1

Thus, $V_1$ shall represent a $\mathbb{Q}$-vector space of dimension $1$.
Likewise, we can use the command `VectorSpaceObject` to create vector space
objects of any finite dimension. What happens if we create another vector
space object of the same dimension?

In [9]:
W1 = CAP.VectorSpaceObject( 1, QQ )
CAP.IsEqualForObjects( V1, W1 )

true

Two objects with the same dimension are considered to be equal in
$\mathrm{mat}_{\mathbb{Q}}$. Since the only datum needed for the
creation of an object was a natural number (including zero),
we can summarize:

$$\mathrm{Obj}\big( \mathrm{mat}_{\mathbb{Q}} \big) = \mathbb{N}_0$$

Next, we turn to **morphisms**.
Given two objects
$$
m,n \in \mathrm{Obj}\big( \mathrm{mat}_{\mathbb{Q}} \big) = \mathbb{N}_0
$$
the set of homomorphisms is simply given by
$$
\mathrm{Hom}_{\mathrm{mat}_{\mathbb{Q}}}( m, n ) := \mathbb{Q}^{m \times n},
$$
i.e., $m$ by $n$ matrices with entries in $\mathbb{Q}$.

The corresponding constructor is called `VectorSpaceMorphism`.

In [10]:
V2 = CAP.VectorSpaceObject( 2, QQ )
mat = CAP.HomalgMatrix( @gap([1, 1, 1, 1]), 2, 2, QQ )
α = CAP.VectorSpaceMorphism( V2, mat, V2 )
GAP.Display( α )

[ [  1,  1 ],
  [  1,  1 ] ]

A morphism in Category of matrices over Q



Please note that the `Display` command is not in the `CAP` module, but in the `GAP` module. It is a special display command for Jupyter notebooks.

The `@gap` macro translates to literal GAP input here, i.e., we can use it to create literal GAP input directly from Julia

In [11]:
@gap [1 .. 5]

GAP: [ 1 .. 5 ]

It only works for literals, not anything that has a variable in it. For translating julia code to GAP, we can use the `julia_to_gap` function

In [12]:
list = [1 2; 3 4]
GAP.julia_to_gap(list)

GAP: [ [ 1, 2 ], [ 3, 4 ] ]

In [13]:
λ = CAP.VectorSpaceMorphism( V1, CAP.HomalgMatrix( @gap([7 , -2]), 1, 2, QQ ), V2 )
GAP.Display( λ )

[ [   7,  -2 ] ]

A morphism in Category of matrices over Q



In [14]:
ρ = CAP.VectorSpaceMorphism( V2, CAP.HomalgMatrix( @gap([-3, 9]), 2, 1, QQ ), V1 )
GAP.Display( ρ )

[ [  -3 ],
  [   9 ] ]

A morphism in Category of matrices over Q



Composition in $\mathrm{mat}_{\mathbb{Q}}$ is given by matrix multiplication.

There are two commands for composition:
`PreCompose` and `PostCompose`

In [15]:
GAP.Display( CAP.PreCompose( λ, ρ ) )

[ [  -39 ] ]

A morphism in Category of matrices over Q



In [16]:
GAP.Display( CAP.PostCompose( λ, ρ ) )

[ [  -21,    6 ],
  [   63,  -18 ] ]

A morphism in Category of matrices over Q



**Exercise 1**

> Create a *non-zero* morphism $\beta: V_2 \rightarrow V_2$ in $\mathrm{mat}_{\mathbb{Q}}$
such that both
$$
\alpha \cdot \beta = 0
$$
and
$$
\alpha \circ \beta = 0\text{,}
$$
where $\alpha = \begin{pmatrix} 1 & 1 \\ 1 & 1\end{pmatrix}: V_2 \rightarrow V_2$.

In [17]:
# Solution of Exercise 1:

# \beta := 

**Identities** in $\mathrm{mat}_{\mathbb{Q}}$ are given by
identity matrices.

In [18]:
GAP.Display( CAP.IdentityMorphism( V2 ) )

[ [  1,  0 ],
  [  0,  1 ] ]

An identity morphism in Category of matrices over Q



Classically, the category of finite dimensional vector spaces
is given by the class of objects $\mathrm{Obj}\big( \mathrm{vec}_{\mathbb{Q}} \big)$ 
of finite dimensional $\mathbb{Q}$-vector spaces,
and homomorphism sets
$\mathrm{Hom}_{ \mathrm{vec}_{\mathbb{Q}} }( V, W )$
given by $\mathbb{Q}$-linear maps $\alpha: V \rightarrow W$
for $\mathbb{Q}$-vector spaces $V, W$.

For us, the crucial point is that the mappings
$$
m \mapsto \mathbb{Q}^m
$$
and
$$
(a_{ij})_{\substack{i = 1, \dots m \\ j = 1, \dots n}} \mapsto
\big( \mathbb{Q}^m \stackrel{(a_{ij})_{ij}}{\longrightarrow} \mathbb{Q}^n \big)
$$
define a functor
$$
\mathrm{mat}_{\mathbb{Q}} \longrightarrow \mathrm{vec}_{\mathbb{Q}}
$$
that is an equivalence of categories.

Thus, simply regarded as categories,
$\mathrm{mat}_{\mathbb{Q}}$ and $\mathrm{vec}_{\mathbb{Q}}$
are indistinguishable.

Of course, this statement can only be computationally useful if 
there are interesting computations to do that only concern the
categorical structure of $\mathrm{vec}_{\mathbb{Q}}$.
Luckily, there is a lot that we can compute, and this is
due to the various constructions provided by the
concept of an *abelian category*.

### Ab-categories

An Ab-category (which is much weaker than an abelian category)
is a category in which all homomorphism sets are abelian groups,
and composition distributes over addition.

In [19]:
GAP.Display( α + α )

[ [  2,  2 ],
  [  2,  2 ] ]

A morphism in Category of matrices over Q



In [20]:
GAP.Display( -α )

[ [  -1,  -1 ],
  [  -1,  -1 ] ]

A morphism in Category of matrices over Q



In [21]:
GAP.Display( CAP.ZeroMorphism( V1, V2 ) )

[ [  0,  0 ] ]

A zero morphism in Category of matrices over Q



In [22]:
γ = CAP.VectorSpaceMorphism( V1, CAP.HomalgMatrix( @gap([3,5]), 1, 2, QQ ), V2 )
δ = CAP.VectorSpaceMorphism( V1, CAP.HomalgMatrix( @gap([-9, 0]), 1, 2, QQ ), V2 )

CAP.IsCongruentForMorphisms(
CAP.PreCompose( γ + δ, α ),
CAP.PreCompose( γ, α ) + CAP.PreCompose( δ, α )
)

true

The command for comparing morphisms on a mathematical level is `IsCongruentForMorphisms`.

*Remark: there is also a command called `IsEqualForMorphisms` that compares
morphisms on a technical level. We will see the difference between
these two notions in our discussion
of the category $\mathrm{fpres}_R$.*

Given two morphisms $\alpha, \beta: A \rightarrow B$, we will write
$$
\alpha \sim_{A,B} \beta
$$
or simply
$$
\alpha \sim \beta
$$
when we mean that they are congruent, i.e., equal on a mathematical level.

### Direct sums

The following description is taken from the CAP manual:

> For a given list $D = (S_1, \dots, S_n)$ in an Ab-category, a direct sum consists of five parts:
- an object $S$,
- a list of morphisms $\pi = (\pi_i: S \rightarrow S_i)_{i = 1 \dots n}$,
- a list of morphisms $\iota = (\iota_i: S_i \rightarrow S)_{i = 1 \dots n}$
- a  dependent  function  $u_{\mathrm{in}}$  mapping  every  list  $\tau  =  (  \tau_i:  T  \rightarrow  S_i  )_{i  =  1 \dots n}$ 
to a morphism $u_{\mathrm{in}}(\tau): T \rightarrow S$ such that $\pi_i \circ u_{\mathrm{in}}(\tau) \sim_{T,S_i} \tau_i$ for all $i = 1, \dots, n$.
- a  dependent  function  $u_{\mathrm{out}}$  mapping  every  list  $\tau  =  (  \tau_i:  S_i  \rightarrow  T  )_{i  = 1 \dots n}$ to a morphism $u_{\mathrm{out}}(\tau): S \rightarrow T$ such that $u_{\mathrm{out}}(\tau) \circ \iota_i \sim_{S_i, T} \tau_i$ for all $i = 1, \dots, n$,
>
>such that
>
> - $\sum_{i=1}^{n} \iota_i \circ \pi_i \sim_{S,S} \mathrm{id}_S$,
- $\pi_j \circ \iota_i \sim_{S_i,S_j} \delta_{i,j}$,
>
> where  $\delta_{i,j}  \in  \mathrm{Hom}(  S_i,  S_j  )$ is the identity if $i=j$, and $0$ otherwise. The $5$-tuple $(S, \pi, \iota, u_{\mathrm{in}},
  u_{\mathrm{out}})$ is called a direct sum of $D$. We denote the object $S$ of such a $5$-tuple by $\bigoplus_{i=1}^n S_i$. We say that the morphisms
  $u_{\mathrm{in}}(\tau),  u_{\mathrm{out}}(\tau)$ are induced by the universal property of the direct sum.

We show how the described functionalities of direct sums
can be addressed in CAP:

The command `DirectSum` takes as an input 
a list of objects $[S_1, \dots, S_n]$
and computes a direct sum object
$\bigoplus_{i=1}^n S_i$:

In [23]:
D = GAP.julia_to_gap([ V1, V2, V2, V1 ])
S = CAP.DirectSum( D )

GAP: A vector space object over Q of dimension 6

Note that the translation of $[ V_1, V_2, V_2, V_1 ]$ to a GAP list is not necessary here, as we can also call `DirectSum` with arbitrary many arguments. However, for the next command, it is necessary.

In [24]:
π₁ = CAP.ProjectionInFactorOfDirectSum( D, 2 )
GAP.Display( π₁ )

[ [  0,  0 ],
  [  1,  0 ],
  [  0,  1 ],
  [  0,  0 ],
  [  0,  0 ],
  [  0,  0 ] ]

A morphism in Category of matrices over Q



Note that the argument in
`ProjectionInFactorOfDirectSum`
is the list $D$, not the already constructed object $S$.
The object $S$ is simply given by the number $6$ and
does not know anything about the way it was created.

In [25]:
ι₁ = CAP.InjectionOfCofactorOfDirectSum( D, 2 )
GAP.Display( ι₁ )

[ [  0,  1,  0,  0,  0,  0 ],
  [  0,  0,  1,  0,  0,  0 ] ]

A morphism in Category of matrices over Q



In [26]:
CAP.IsCongruentForMorphisms(
    CAP.IdentityMorphism( V2 ),
    CAP.PreCompose( ι₁, π₁ )
)

true

In [27]:
GAP.Display( 
    CAP.UniversalMorphismIntoDirectSum( GAP.julia_to_gap([
        CAP.ZeroMorphism( V2, V1 ),
        α,
        -3 * CAP.IdentityMorphism( V2 ),
        CAP.ZeroMorphism( V2, V1 )
    ] ) )
)

[ [   0,   1,   1,  -3,   0,   0 ],
  [   0,   1,   1,   0,  -3,   0 ] ]

A morphism in Category of matrices over Q



In [28]:
GAP.Display( 
    CAP.UniversalMorphismFromDirectSum( GAP.julia_to_gap([
        CAP.ZeroMorphism( V1, V2 ),
        α,
        -3 * CAP.IdentityMorphism( V2 ),
        CAP.ZeroMorphism( V1, V2 )
        ]) )
)

[ [   0,   0 ],
  [   1,   1 ],
  [   1,   1 ],
  [  -3,   0 ],
  [   0,  -3 ],
  [   0,   0 ] ]

A morphism in Category of matrices over Q



**Exercise 2**
> Create the morphism
> $$\alpha = \begin{pmatrix} 1 & 1 \\ 1 & 1\end{pmatrix}: V_2 \rightarrow V_2$$ 
> 
> only using the object $V_1$ and the following commands:
> - `IdentityMorphism`
> - `UniversalMorphismFromDirectSum`
> - `UniversalMorphismIntoDirectSum`
 

In [29]:
# Solution of Exercise 2:



### Kernels

Given a morphism 

$$
\mu: V \rightarrow W
$$

between vector spaces $V,W$, the common
way to describe its kernel set-theoretically
is given by

$$
\mathrm{kernel}(\mu) := \{ v \in V \mid \mu(v) = 0 \}.
$$

Clearly, this definition makes no sense in our computational
model $\mathrm{mat}_{\mathbb{Q}}$, where we do not have
access to "elements of vector spaces", since objects in
$\mathrm{mat}_{\mathbb{Q}}$ are
simply represented by natural numbers.

**Nevertheless, category theory provides enough language
that lets us work with kernels anyway.**


The following description is taken from the CAP manual:

>For a given morphism $\alpha: A \rightarrow B$, a kernel of $\alpha$ consists of three parts:
- an object $K$,
- a morphism $\iota: K \rightarrow A$ such that $\alpha \circ \iota \sim_{K,B} 0$,
- a  dependent  function $u$ mapping each morphism $\tau: T \rightarrow A$ satisfying $\alpha \circ \tau \sim_{T,B} 0$ to a morphism $u(\tau): T \rightarrow K$ such that $\iota\circ u( \tau ) \sim_{T,A} \tau$.
  >
>  The  triple  $( K, \iota, u )$ is called a kernel of $\alpha$ if the morphisms $u( \tau )$ are uniquely determined up to congruence of morphisms. We denote the object $K$ of such a
  triple  by  $\mathrm{KernelObject}(\alpha)$. We  say  that the morphism $u(\tau)$ is induced by the universal property of the kernel.
  
  
We show how the described functionalities of kernels
can be addressed in CAP:


Recall that $\alpha$ was given by

In [30]:
GAP.Display( α )

[ [  1,  1 ],
  [  1,  1 ] ]

A morphism in Category of matrices over Q



In [31]:
K = CAP.KernelObject( α )

GAP: A vector space object over Q of dimension 1

In [32]:
ι = CAP.KernelEmbedding( α )
GAP.Display( ι )

[ [  -1,   1 ] ]

A split monomorphism in Category of matrices over Q



So, instead of "seeing" the linear dependencies of the rows
of the underlying matrix of $\alpha$ within the kernel object,
we can "see" them in the kernel embedding of $\alpha$ instead.

Next, we create a test morphism.

In [33]:
τ = CAP.UniversalMorphismFromDirectSum( GAP.julia_to_gap([ 2*ι, ι ]) )
GAP.Display( τ )

[ [  -2,   2 ],
  [  -1,   1 ] ]

A morphism in Category of matrices over Q



From the induced map of the universal property of the kernel,
we can read off how the rows of $\tau$ are given as linear
combinations of the rows in the kernel embedding.

In [34]:
lift = CAP.KernelLift( α, τ )
GAP.Display( lift )

[ [  2 ],
  [  1 ] ]

A morphism in Category of matrices over Q



In [35]:
CAP.IsCongruentForMorphisms(
    CAP.PreCompose( lift, ι ),
    τ
)

true

*Remark: on a technical level, Gaussian elemination is triggered in order to 
compute the outputs of the commands given above. However, as long as you can phrase
and interpret the entities that you wish to compute categorically, you do not need to know
the technicalities behind the commands.*

**Exercise 3**
> Write a function for the functoriality of the kernel, i.e., a function that takes as an input
> $$\mu_1: A \rightarrow B$$
> $$\mu_2: A' \rightarrow B'$$
> $$\nu_1: A \rightarrow A'$$
> $$\nu_2: B \rightarrow B'$$
> such that
> $$\mu_1 \cdot \nu_2 \sim \nu_1 \cdot \mu_2\text{,}$$
> and gives as an output a morphism
> $$\iota: \mathrm{KernelObject}( \mu_1 ) \longrightarrow \mathrm{KernelObject}( \mu_2 )$$
> such that
> $$\iota \cdot \mathrm{KernelEmbedding}( \mu_2 ) \sim \mathrm{KernelEmbedding}( \mu_1 ) \cdot \nu_1\text{.}$$
> 
> Only use those CAP commands that you have already seen in this worksheet.

In [36]:
# Solution of Exercise 3
function kernelfunctorial(μ₁,μ₂,ν₁,ν₂)
   ## Your code goes here 
end

kernelfunctorial (generic function with 1 method)

### Cokernels

In order to properly introduce the cokernel of a morphism

$$
\mu: V \rightarrow W
$$

between vector spaces $V,W$ in a set-theoretical way,
we would need to introduce the concept of images and quotient
spaces first.

In the language of category theory, we can simply state
that the definition of a cokernel is dual to the definition
of a kernel, i.e., simply reverse all the arrows in 
the categorical definition of the kernel.

The following description is taken from the CAP manual:

> For a given morphism $\alpha: A \rightarrow B$, a cokernel of $\alpha$ consists of three parts:
  - an object $K$,
  - a morphism $\epsilon: B \rightarrow K$ such that $\epsilon \circ \alpha \sim_{A,K} 0$,
  - a  dependent  function  $u$ mapping each $\tau: B \rightarrow T$ satisfying $\tau \circ \alpha \sim_{A, T} 0$ to a morphism $u(\tau):K \rightarrow T$ such that $u(\tau) \circ \epsilon \sim_{B,T} \tau$.
>
> The  triple  $(  K,  \epsilon, u )$ is called a cokernel of $\alpha$ if the morphisms $u( \tau )$ are uniquely determined up to congruence of morphisms. We denote the object $K$ of
  such  a  triple  by $\mathrm{CokernelObject}(\alpha)$. We say that the morphism $u(\tau)$ is induced by the universal property of the cokernel.
  
We show how the described functionalities of cokernels
can be addressed in CAP:

Recall that $\alpha$ was given by

In [37]:
GAP.Display( α )

[ [  1,  1 ],
  [  1,  1 ] ]

A morphism in Category of matrices over Q



In [38]:
C = CAP.CokernelObject( α )

GAP: A vector space object over Q of dimension 1

In [39]:
ϵ = CAP.CokernelProjection( α )
GAP.Display( ϵ )

[ [  -1 ],
  [   1 ] ]

A split epimorphism in Category of matrices over Q



In [40]:
τ = ϵ
colift = CAP.CokernelColift( α, τ )
GAP.Display( colift )

[ [  1 ] ]

A morphism in Category of matrices over Q



In [41]:
CAP.IsCongruentForMorphisms(
    CAP.PreCompose( ϵ, colift ),
    τ
)

true

**Exercise 4**

> Construct your own example of morphisms
>
> $$\sigma, \tau$$
>
> such that
> `CokernelColift( sigma, tau )` is neither the identity nor zero.

In [42]:
# Solution of Exercise 4



### Lift along monos, colift along epis

The definition of an abelian category involves the following two statements:
> every mono is the kernel of its cokernel

and
> every epi is the cokernel of its kernel.

Unwrapping these shortcut statements, we see that
we have to able to compute
> lifts along monomorphisms

and
> colifts along epimorphisms.

We start with lifts along monomorphisms.

In [43]:
mono =
    CAP.VectorSpaceMorphism( V1, CAP.HomalgMatrix( @gap([1,-1]), 1, 2, QQ ), V2 )
GAP.Display( mono )

[ [   1,  -1 ] ]

A morphism in Category of matrices over Q



In [44]:
CAP.IsMonomorphism( mono )

true

In [45]:
τ = CAP.VectorSpaceMorphism( V2, CAP.HomalgMatrix( @gap([2,-2, 3,-3]), 2, 2, QQ ), V2 )
GAP.Display( τ )

[ [   2,  -2 ],
  [   3,  -3 ] ]

A morphism in Category of matrices over Q



In [46]:
CAP.IsZeroForMorphisms( 
    CAP.PreCompose( τ, CAP.CokernelProjection( mono ) )
)

true

Since we expect `mono` to be the kernel embedding of its cokernel projection,
and since `tau` is a proper test morphism, we should be able
to compute the lift:

In [47]:
lift = CAP.LiftAlongMonomorphism( mono, τ )
GAP.Display( lift )

[ [  2 ],
  [  3 ] ]

A morphism in Category of matrices over Q



**Exercise 5**
> The command for colifts along epimorphism is given by
> `ColiftAlongEpimorphism`. Create an example for its usage.

In [48]:
# Solution of Exercise 5



### Category of finitely presented left modules

As promised, we introduce a computable
model of the category $\mathrm{mod}_R$ of finitely presented (left)
modules over a computable ring $R$.
A **(left) computable ring** is a ring $R$ 
equipped with

- an algorithm to compute a finite generating set of the row syzygies of a given matrix,
- an algorithm to decide the existence and to find a particular solution of a left-sided inhomogeneous linear system, i.e., a linear system of the form $X \cdot A = B$ for given matrices $A,B$ over $R$.

*Remark: the existence of finite generating sets of row syzygies is actually
equivalent to the category $\mathrm{mod}_R$
being abelian.*

We are going to create the ring of integers
$$
R := \mathbb{Z}.
$$

In [49]:
R = CAP.HomalgRingOfIntegers()

GAP: Z

*Convention: when we say module, we will always mean a left module.*

An $R$-module $M$ is finitely presented if
$$ \exists m,n \in \mathbb{N}_0 \exists (a_{ij})_{ij} \in R^{m \times n}: M \simeq \mathrm{CokernelObject}( R^m \stackrel{(a_{ij})_{ij}}{\longrightarrow} R^n ) $$

Thus, matrices with entries in $R$ are great candidates for a data structure
of the objects in our computational model $\mathrm{fpres}_R$.

Our computational model $\mathrm{fpres}_R$ for finitely presented (left) modules,
which we call the **category of left presentations** of $R$, is part of the following $\textsf{GAP}$-package:

In [50]:
LoadPackageAndExposeGlobals( "ModulePresentationsForCAP", CAP )

In [51]:
fpresR = CAP.LeftPresentations( R )

GAP: Category of left presentations of Z

To construct an object in $\mathrm{fpres}_R$, we use the following constructor:

In [52]:
M1 = CAP.AsLeftPresentation( CAP.HomalgMatrix( @gap([1,2,1,1]), 2, 2, R ) )
GAP.Display( M1 )

[ [  1,  2 ],
  [  1,  1 ] ]

An object in Category of left presentations of Z



In [53]:
M2 = CAP.AsLeftPresentation( CAP.HomalgIdentityMatrix( 2, R ) )
GAP.Display( M2 )

[ [  1,  0 ],
  [  0,  1 ] ]

An object in Category of left presentations of Z



The two objects $M_1, M_2$ that we have just created
are considered to be unequal, since their underlying
matrices are unequal:

In [54]:
CAP.IsEqualForObjects( M1, M2 )

false

However, both objects are isomorphic to the zero object:

In [55]:
CAP.IsZeroForObjects( M1 )

true

In [56]:
CAP.IsZeroForObjects( M2 )

true

This computation reflects the fact that the matrix over $R$
presenting a module $M$ is not uniquely determined.
Nevertheless, this does not prevent us from taking such matrices
as a data structure for the objects in $\mathrm{fpres}_R$,
as long as our choice for the morphisms will grantuee that we
will end up with an equivalence
$$
\mathrm{mod}_R \simeq \mathrm{fpres}_R.
$$

A morphism between two objects
$$
(a_{ij})_{ij}, (b_{kl})_{kl} \in \mathrm{Obj}\big( \mathrm{fpres}_R \big)
$$
is given by a matrix
$$
(c_{jl})_{jl}
$$
over $R$ such that there exists another matrix $( d_{ik} )_{ik}$ over $R$
such that
$$
(a_{ij})_{ij} \cdot (c_{jl})_{jl} = (d_{ik})_{ik} \cdot (b_{kl})_{kl}\text{.}
$$

As an example, let us create the natural epimorphism
$$
\epsilon: R \twoheadrightarrow R/\langle 2 \rangle
$$

In [57]:
F = CAP.FreeLeftPresentation( 1, R )
GAP.Display( F )

(an empty 0 x 1 matrix)

An object in Category of left presentations of Z



In [58]:
M = CAP.AsLeftPresentation( CAP.HomalgMatrix( @gap([ 2 ]), 1, 1, R ) )
GAP.Display( M )

[ [  2 ] ]

An object in Category of left presentations of Z



In [59]:
ϵ = CAP.PresentationMorphism(
    F,
    CAP.HomalgIdentityMatrix( 1, R ),
    M
)
GAP.Display( ϵ )

[ [  1 ] ]

A morphism in Category of left presentations of Z



To check that the existence statement in the definition of 
a morphism in $\mathrm{fpres}_R$ is satisfied, we can 
call the following command:

In [60]:
CAP.IsWellDefined( ϵ )

true

A quicker way to create $\epsilon$ is the following:

In [61]:
GAP.Display( CAP.CoverByFreeModule( M ) )

[ [  1 ] ]

A morphism in Category of left presentations of Z



Mathematically, the following matrix gives also rise to the morphism $\epsilon$:

In [62]:
ϵ₂ = CAP.PresentationMorphism(
    F,
    CAP.HomalgMatrix( @gap([3]), 1, 1, R ),
    M
)
GAP.Display( ϵ₂ )

[ [  3 ] ]

A morphism in Category of left presentations of Z



In [63]:
CAP.IsCongruentForMorphisms( ϵ, ϵ₂ )

true

However, the underlying matrices defining $\epsilon$
and $\epsilon_2$ differ, which is what is asked for
when we use the following command:

In [64]:
CAP.IsEqualForMorphisms( ϵ, ϵ₂ )

false

So remember:
`IsCongruentForMorphisms` is the mathematical notion of equality,
`IsEqualForMorphisms` is the technical notion of equality.
The latter is used for the technical specifications of CAP-operations:
> "equal" input yields "equal" output,

where the equality notion is always meant to be the technical one.

It is mainly due to the comparision theorem of projective resolutions
and the functoriality of cokernels
that the map
$$
(a_{ij})_{\substack{i = 1, \dots m \\ j = 1, \dots n}} \mapsto
\mathrm{CokernelObject}\big( {R}^m \stackrel{(a_{ij})_{ij}}{\longrightarrow} {R}^n \big)
$$
extends to the level of morphisms and defines a functor
$$
\mathrm{fpres}_{R} \longrightarrow \mathrm{mod}_{R}
$$
that is an equivalence of categories.

Moreover, $\mathrm{mod}_{R}$
is an abelian category and consequently so is $\mathrm{fpres}_{R}$.
We can use **exactly the same commands** as in the case of $\mathrm{mat}_k$
in order to operate in $\mathrm{fpres}_{R}$.

Here are examples:

In [65]:
GAP.Display( CAP.KernelEmbedding( ϵ ) )

[ [  2 ] ]

A monomorphism in Category of left presentations of Z



In [66]:
CAP.IsZeroForObjects( CAP.CokernelObject( ϵ ) )

true

In [67]:
GAP.Display( CAP.DirectSum( F, M, M, F ) )

[ [  0,  2,  0,  0 ],
  [  0,  0,  2,  0 ] ]

An object in Category of left presentations of Z



**Exercise 6**
> Create a free resolution of $R/\langle \begin{pmatrix} 1 & 2 & 3 \\ 4 & 5 & 6 \\ 7 & 8 & 9 \end{pmatrix} \rangle$ in CAP.

In [68]:
# Solution of Exercise 6



### Generic algorithms for abelian categories

Our computational models
$\mathrm{mat}_k$ and $\mathrm{fpres}_R$
are both examples of abelian categories.
Let us refer to a CAP operation
that can be interpreted
in any abelian category
as an **abelian operation**.
It then follows that any $\mathsf{GAP}$ function
that we build up only using abelian operations
can be executed both in $\mathrm{mat}_k$ and $\mathrm{fpres}_R$.

As an example, try out the code you wrote
to solve Exercise 3 on the following input:

In [69]:
M = CAP.AsLeftPresentation( CAP.HomalgMatrix( @gap([ [ 2, 2, 2 ], [ 3, 0, 0 ], [ 0, 7, 0 ] ]), 3, 3, R ) )
N = CAP.AsLeftPresentation( CAP.HomalgMatrix( @gap([ [ 2, 2, 2 ], [ 3, 0, 0 ] ]), 2, 3, R ) )
μ₁ = CAP.CoverByFreeModule( N )
μ₂ = CAP.CoverByFreeModule( M )
ν₁ = CAP.PresentationMorphism( CAP.Source( μ₁ ), CAP.HomalgIdentityMatrix( 3, R ), CAP.Source( μ₂ ) )
ν₂ = CAP.PresentationMorphism( N, CAP.HomalgIdentityMatrix( 3, R ), M )

GAP: A morphism in Category of left presentations of Z

Did you write your code generically enough that it was not only applicable in the context
of $\mathrm{mat}_k$, but also in $\mathrm{fpres}_R$?

**Exercise 7**
> Write generic algorithms for
> 1. the intersection $A \cap B$ of two subobjects $A,B \hookrightarrow C$
> 2. the sum $A + B$ of two subobjects $A,B \hookrightarrow C$ 
>
> that work in any abelian category and only uses CAP
> commands that you have seen so far in this worksheet.
> Note that subobjects of an object $C$ are simply modeled by monomorphisms into $C$.
> Test your code on
> the following examples:

**Test case 1**

Category: $\mathrm{mat}_{\mathbb{Q}}$

In [70]:
C = CAP.VectorSpaceObject( 3, QQ )

A = CAP.VectorSpaceMorphism(
    CAP.VectorSpaceObject( 2, QQ ),
    CAP.HomalgMatrix( @gap([1,1,0, 0,1,1]), 2, 3, QQ ), 
    C
)

B = CAP.VectorSpaceMorphism(
    CAP.VectorSpaceObject( 2, QQ ),
    CAP.HomalgMatrix( @gap([1,0,1, 1,1,0]), 2, 3, QQ ), 
    C
)

GAP.Display( A )
GAP.Display( B )

[ [  1,  1,  0 ],
  [  0,  1,  1 ] ]

A morphism in Category of matrices over Q

[ [  1,  0,  1 ],
  [  1,  1,  0 ] ]

A morphism in Category of matrices over Q



**Test case 2**

Category: $\mathrm{fpres}_{R}$

In [71]:
A = CAP.AsMorphismBetweenFreeLeftPresentations(
    CAP.HomalgMatrix( @gap([42]), 1, 1, R )
)

B = CAP.AsMorphismBetweenFreeLeftPresentations(
    CAP.HomalgMatrix( @gap([63]), 1, 1, R )
)

GAP.Display( A )
GAP.Display( B )

[ [  42 ] ]

A morphism in Category of left presentations of Z

[ [  63 ] ]

A morphism in Category of left presentations of Z



In [72]:
# Solution of Exercise 7

